In [ ]:
!pip install funcy
!pip install bs4
!pip install scikit-learn
!pip install imagesize
!pip install tqdm

In [ ]:
!pip install layoutparser

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

In [ ]:
import torch
print(torch.__version__)
if torch.cuda.is_available():
    print("CUDA is available.")
    device = torch.cuda.get_device_name(0)
    print("GPU Device:", device)
else:
    print("CUDA is not available.")

In [ ]:
! wget https://layoutlm.blob.core.windows.net/docbank/dataset/DocBank_500K_ori_img.zip.001

%mv /kaggle/working/DocBank_500K_ori_img.zip.001 /kaggle/working/images.zip

import os 
!7z x /kaggle/working/images.zip

# need docbankjsonfile

In [ ]:
!git clone https://github.com/Layout-Parser/layout-model-training.git

In [ ]:
!python -m pip install git+https://github.com/facebookresearch/detectron2.git

In [ ]:
from pycocotools.coco import COCO
import layoutparser as lp
import random
import cv2

def load_coco_annotations(annotations, coco=None):
    
    layout = lp.Layout()

    for ele in annotations:

        x, y, w, h = ele['bbox']

        layout.append(
            lp.TextBlock(
                block = lp.Rectangle(x, y, w+x, h+y),
                type  = ele['category_id'] if coco is None else coco.cats[ele['category_id']]['name'],
                id = ele['id']
            )
        )

    return layout

In [ ]:
COCO_ANNO_PATH = '/kaggle/input/datasetv15/docs.json'
COCO_IMG_PATH  = '/kaggle/working/images'

coco = COCO(COCO_ANNO_PATH)

In [ ]:
for image_id in random.sample(coco.imgs.keys(), 2):
    image_info = coco.imgs[image_id]
    annotations = coco.loadAnns(coco.getAnnIds([image_id]))

    image = cv2.imread(f'{COCO_IMG_PATH}/{image_info["file_name"]}')
    layout = load_coco_annotations(annotations, coco)

    viz = lp.draw_box(image[...,::-1], layout)
    display(viz) # show the results

In [ ]:
!python /content/layout-model-training/utils/cocosplit.py \
  --annotation-path       ../kaggle/input/datasetv15/docs.json \
  --split-ratio           0.85 \
  --train               ../kaggle/working/train.json \
  --test                ../kaggle/working/test.json

In [ ]:
!python /content/layout-model-training/tools/train_net.py \
    --dataset_name          docs-dataset \
    --json_annotation_train ../kaggle/working/train.json \
    --image_path_train      ../kaggle/working/images \
    --json_annotation_val   ../kaggle/working/test.json \
    --image_path_val        ../kaggle/working/images \
    --config-file           ../content/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml \
    OUTPUT_DIR  ../content/output/x101.yaml \
    SOLVER.IMS_PER_BATCH 2